In [1]:
from dotenv import load_dotenv
import os

from langchain.document_loaders import UnstructuredFileLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_groq import ChatGroq
from langchain.chains import RetrievalQA



load_dotenv()

True

In [3]:
# laod api key
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

In [4]:
# load the documents
loader = UnstructuredFileLoader("iitm_bs_degree_student_handbook.pdf")
documents = loader.load()

d:\learn-genai\chatty\rag-llm\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
#print(documents)

In [6]:
# split text to fix exceed embedding length issue
text_splitter = CharacterTextSplitter(
    chunk_size = 2000, 
    chunk_overlap = 400
    ) # overlap necessary

In [7]:
#split text docs
texts = text_splitter.split_documents(documents)

In [8]:
# create embeedings
embeddings = HuggingFaceEmbeddings()

d:\learn-genai\chatty\rag-llm\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


In [9]:
# create a database director to store chroma db embeddings
db_dir = "vector_db"

In [10]:
# create a vector db
vector_db = Chroma.from_documents(
    documents = texts,
    embedding = embeddings, #hugging face
    persist_directory=db_dir # directory to store files
)

# creates a directory

In [11]:
# add llm
llm = ChatGroq(
    model = "llama-3.1-70b-versatile",
    temperature = 0
)

In [12]:
# add retriver to find most common vectors using vector similarity search
retreiver = vector_db.as_retriever()

In [13]:
# create a retrival q/a chain
qa_chain = RetrievalQA.from_chain_type(
    llm = llm,
    chain_type = "stuff",
    retriever = retreiver,
    return_source_documents = True
)

In [14]:
# invoke llm with query
query = "Who can enroll in iitm bs degree program" 


In [15]:
# get response back
response = qa_chain.invoke({"query": query})

print(response["result"])
print("---"*3)
print(f'source: {response["source_documents"][0].metadata["source"]}') 

According to the text, anyone who has passed Class 12 or equivalent can apply for the Qualifier exam, which is the first step to enroll in the IITM BS degree program, irrespective of age or academic background. Additionally, school students who have appeared for their Class 11 final exams can also apply, but they can join the program only after passing Class 12. The minimum eligibility is that the applicant should have studied Mathematics and English in Class 10.
---------
source: iitm_bs_degree_student_handbook.pdf
